In [9]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [10]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


In [11]:
CALLERS = ["A", "B", "C"]

@ray.remote
class MethodStateCounter:
    def __init__(self):
        self.invokers = {"A": [0], "B": [0], "C": [0]}
    
    def invoke(self, name):
        # pretend to do some work
        time.sleep(0.5)
        
        # compute a random result and update the result
        result = random.randint(5, 25)
        self.invokers[name].append(result)
        return result
        
    def get_invoker_state(self, name):
        return self.invokers[name]
    
    def get_number_of_calls(self, name):
        return len(self.invokers[name])
    
    def get_all_invoker_state(self):
        return self.invokers

In [12]:
actor_instance = MethodStateCounter.remote()

In [13]:
for _ in range(10):
    name = random.choice(CALLERS)
    actor_instance.invoke.remote(name)

#### 1. Get number of times an invoker name was called

In [14]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_number_of_calls.remote(name))
print(f"Number of times: {name} was called: {count}")

Number of times: B was called: 3


#### 2. Get computed values by invoker name

In [15]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_invoker_state.remote(name))
print(f"All values computed by: {name} is: {count}")

All values computed by: C is: [0, 7, 6, 15, 10]


#### 3. Get state of all invokers

In [16]:
print(f"State of all computed values for invokers: {ray.get(actor_instance.get_all_invoker_state.remote())}")

State of all computed values for invokers: {'A': [0, 9, 24, 11, 10], 'B': [0, 18, 19], 'C': [0, 7, 6, 15, 10]}


In [17]:
ray.shutdown()